In [2]:
from numpy import genfromtxt
import matplotlib.pyplot as plt
#!pip install mplfinance
#import mplfinance as mpf
#from mplfinance.original_flavor import candlestick2_ochl, candlestick_ohlc
import numpy as np
import pandas as pd
#import uuid
import time
import shutil
import os
import sys
from SR import *


#os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
#import keras
import tensorflow
from tensorflow import keras
import tensorflow.keras.models as M
import tensorflow.keras.layers as L
from keras_OneCycle import OneCycle

In [3]:
input_size = 432
n_steps = 10
data = pd.read_csv("data/eurusd/eurusd_H1.csv")

for i in ["r1", "r2", "r3", "s1", "s2", "s3"]:
    data[i] = np.nan


# get index of signals
signals = data.loc[(data["1"] == 1) | (data["1"] == -1)].index

# create training_data based on signals
x_train = [data[s-input_size+1:s+1] for s in signals if s > input_size]


# create HOLD training data
np.random.seed(42)
hold_signals = data.loc[(data["1"] != 1) & (data["1"] != -1)].index
non_signal_index = np.random.choice(data.loc[(data["1"] != 1) & (data["1"] != -1)].index, size=len(signals)//2)
non_signal_index = non_signal_index[non_signal_index > input_size]

x_train_hold = [data[s-input_size+1:s+1] for s in non_signal_index if s > input_size]
x_train = x_train + x_train_hold
y_train = np.append(data["1"][signals][data["1"][signals].index > input_size].values, np.zeros(len(x_train_hold)))
y_train[y_train == -1] = 2  # change sell value to positive, 0=hold, 1=buy, 2=sell

# Support and Resistance

- 3 above and below last close price
- top 3 and bottom 3 of entire range
- all of them

In [4]:
# get 3 above and 3 below last close price

for i, df in enumerate(x_train):
    df = sr(df, input_size).drop(columns=["1", "s", "r"])
    
    # 3 above and 3 below last close price
    levels = np.array(sorted([l for l in df.columns if type(l) == np.float64]))
    last_close = df.close[-1]
    r3 = levels[(levels - last_close) > 0][:3]
    s3 = levels[(levels - last_close) < 0][-3:]
    
    # change resistance names
    for l, name in zip(r3, ["r1", "r2", "r3"]):
        df[name] = df[l]

    # change support names
    for l, name in zip(s3, ["s1", "s2", "s3"]):
        #df.rename(columns = {l : name}, errors="raise", inplace=True)
        df[name] = df[l]


    # remove all other levels columns
    levels = np.array(sorted([l for l in df.columns if type(l) == np.float64]))
    x_train[i] = df.drop(columns=levels).fillna(0)[-100:]

<ipython-input-4-98de902de16a>:9: RuntimeWarning: invalid value encountered in greater
  r3 = levels[(levels - last_close) > 0][:3]
<ipython-input-4-98de902de16a>:10: RuntimeWarning: invalid value encountered in less
  s3 = levels[(levels - last_close) < 0][-3:]


In [5]:
# select which data to use
#data = data[["index","open", "high", "low", "close", "1"]]


# convert from dataframe to numpy array    
for i, df in enumerate(x_train):
    x_train[i] = df.to_numpy()
x_train = np.array(x_train)

In [7]:
# If you like to speed up training process with GPU, first install PlaidML and then uncomment the following line.
# Otherwise it will fallback to tensorflow.

# Enter the number of samples, training + validation
nb_train_samples = round(len(x_train)*.8)
nb_val_samples = round(len(x_train)*.2)

# We have 3 classes, buy sell, and hold
n_features = x_train[0].shape[1]
num_classes = 3
#batch_size = 8
lr = 0.001

def make_model(input_shape):
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=128, kernel_size=3, padding="same", activation="relu")(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Dropout(0.5)(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=3, padding="same", activation="relu")(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Dropout(0.5)(conv2)

    conv3 = keras.layers.Conv1D(filters=512, kernel_size=3, padding="same", activation="relu")(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Dropout(0.5)(conv3)

    gap = keras.layers.GlobalAveragePooling1D()(conv3)

    output_layer = keras.layers.Dense(num_classes, activation="softmax")(gap)

    return keras.models.Model(inputs=input_layer, outputs=output_layer)

def make_model1(input_shape):
    model = M.Sequential()
    model.add(L.Input(shape=input_shape))
    model.add(L.LSTM(200, activation="relu"))
    model.add(L.Dense(num_classes, activation="softmax"))
    
    return model

def make_model2(input_shape):
    model = M.Sequential()
    model.add(L.Input(input_shape))
    model.add(L.Dense(100, activation="selu"))
    model.add(L.BatchNormalization())
    
    model.add(L.Dense(100, activation="selu"))
    model.add(L.BatchNormalization())
    
    model.add(L.Dense(100, activation="selu"))
    model.add(L.BatchNormalization())
    
    model.add(L.Dense(3, activation="softmax"))
    
    return model
    

model = make_model(input_shape=(100,x_train[0].shape[1]))
#keras.utils.plot_model(model, show_shapes=True)
#model.summary()

# Calling `save('my_model')` creates a SavedModel folder `my_model`.
#model.save("my_model")

# It can be used to reconstruct the model identically.
#reconstructed_model = keras.models.load_model("my_model")

In [8]:
%%time
epochs = 500
batch_size = 32

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]

opt = keras.optimizers.Adagrad()
model.compile(
    optimizer=opt,
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=2,
)

Epoch 1/500
19/19 - 8s - loss: 0.9930 - sparse_categorical_accuracy: 0.5449 - val_loss: 1.1888 - val_sparse_categorical_accuracy: 0.2781
Epoch 2/500
19/19 - 9s - loss: 0.8442 - sparse_categorical_accuracy: 0.6744 - val_loss: 1.2998 - val_sparse_categorical_accuracy: 0.1722
Epoch 3/500
19/19 - 8s - loss: 0.8138 - sparse_categorical_accuracy: 0.6811 - val_loss: 1.3119 - val_sparse_categorical_accuracy: 0.1788
Epoch 4/500
19/19 - 7s - loss: 0.7883 - sparse_categorical_accuracy: 0.7060 - val_loss: 1.2912 - val_sparse_categorical_accuracy: 0.1987
Epoch 5/500
19/19 - 8s - loss: 0.7567 - sparse_categorical_accuracy: 0.7043 - val_loss: 1.2937 - val_sparse_categorical_accuracy: 0.2053
Epoch 6/500
19/19 - 8s - loss: 0.7611 - sparse_categorical_accuracy: 0.7043 - val_loss: 1.2575 - val_sparse_categorical_accuracy: 0.2384
Epoch 7/500
19/19 - 9s - loss: 0.7442 - sparse_categorical_accuracy: 0.7126 - val_loss: 1.2363 - val_sparse_categorical_accuracy: 0.2583
Epoch 8/500
19/19 - 10s - loss: 0.7385 - 

KeyboardInterrupt: 

In [9]:
metric = "sparse_categorical_accuracy"
plt.figure()
plt.plot(history.history[metric])
plt.plot(history.history["val_" + metric])
plt.title("model " + metric)
plt.ylabel(metric, fontsize="large")
plt.xlabel("epoch", fontsize="large")
plt.legend(["train", "val"], loc="best")
plt.show()
plt.close()

NameError: name 'history' is not defined

<Figure size 432x288 with 0 Axes>

In [54]:
x_train.shape

(753, 100, 73)